## Unit 13.6 Capstone.
I found a comparable project.  Here is a link to it.<br>
Predicting Bad Housing Loans using Public Freddie Mac Data — a tutorial on working with imbalanced data
https://towardsdatascience.com/predicting-bad-housing-loans-using-public-freddie-mac-data-a-tutorial-on-working-with-imbalanced-d2852c003996
https://github.com/tsofoon/freddie

I attempted to replicate the project.  However, the data is too much to load into my computer.  I kept getting memory error in jupyter notebook.  
Here is part of the code were I loaded the loan origination data into mysql.  The time series data was not able to load into mysql because of memory issue.

Loan Level Data from http://www.freddiemac.com/research/datasets/sf_loanlevel_dataset.page

In [1]:
# Read database credentials from file
import configparser
config = configparser.ConfigParser()
config.read('mysql.properties')

host=config.get('DatabaseSection', 'database.host')
dbname=config.get('DatabaseSection', 'database.dbname')
user=config.get('DatabaseSection', 'database.user')
pwd=config.get('DatabaseSection', 'database.password')

In [2]:
import pandas as pd
import numpy as np

## Load Origination Data File into Database
loans and remittance table should be empty before loading

In [3]:
# Setup mysql db connection
from sqlalchemy import create_engine
import pymysql

sqlEngine = create_engine(f'mysql+pymysql://{user}:{pwd}@{host}/{dbname}', pool_recycle=3600)
con = sqlEngine.connect()

In [4]:
def loadDataFile(fileName, tableName, con, columns, data_type):
    df = pd.read_csv('data/' + fileName, delimiter='|', header=None, names=columns, dtype = data_type)
    if tableName == 'loans':
        df['NUMBER_OF_UNITS'] = df['NUMBER_OF_UNITS'].str.strip()
        df['NUMBER_OF_UNITS'] = np.where(df['NUMBER_OF_UNITS'] == '.', '99', df['NUMBER_OF_UNITS'])
        df.to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
    elif tableName == 'remittance':
        df['REMAINING_MONTHS_TO_MATURITY'] = df['REMAINING_MONTHS_TO_MATURITY'].str.strip()
        df['REMAINING_MONTHS_TO_MATURITY'] = np.where(df['REMAINING_MONTHS_TO_MATURITY'] == '.', None, df['REMAINING_MONTHS_TO_MATURITY'])
        df['REMAINING_MONTHS_TO_MATURITY'] = np.where(df['REMAINING_MONTHS_TO_MATURITY'] == '***', None, df['REMAINING_MONTHS_TO_MATURITY'])
        df['CURRENT_DEFERRED_UPB'] = np.where(df['CURRENT_DEFERRED_UPB'] == '.', None, df['CURRENT_DEFERRED_UPB'])
        df['NET_SALES_PROCEEDS'] = np.where(df['NET_SALES_PROCEEDS'] == 'U', None, df['NET_SALES_PROCEEDS'])
        # need to split up the write as it overwhelm the laptop
        df[df['CURRENT_ACTUAL_UPB']>=200000].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=180000) & (df['CURRENT_ACTUAL_UPB']<200000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=170000) & (df['CURRENT_ACTUAL_UPB']<180000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=150000) & (df['CURRENT_ACTUAL_UPB']<170000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=130000) & (df['CURRENT_ACTUAL_UPB']<150000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=120000) & (df['CURRENT_ACTUAL_UPB']<130000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=110000) & (df['CURRENT_ACTUAL_UPB']<120000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=100000) & (df['CURRENT_ACTUAL_UPB']<110000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=90000) & (df['CURRENT_ACTUAL_UPB']<100000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=80000) & (df['CURRENT_ACTUAL_UPB']<90000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=70000) & (df['CURRENT_ACTUAL_UPB']<80000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=60000) & (df['CURRENT_ACTUAL_UPB']<70000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=50000) & (df['CURRENT_ACTUAL_UPB']<60000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=40000) & (df['CURRENT_ACTUAL_UPB']<50000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']>=30000) & (df['CURRENT_ACTUAL_UPB']<40000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        df[(df['CURRENT_ACTUAL_UPB']<30000)].to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
        # df.to_sql(name=tableName, con=con, schema='mortgage', if_exists='append', index=False)
    return df

In [5]:
originationFiles = ['historical_data_excl_', 'historical_data_'] # 1999Q1

orig_columns=['CREDIT_SCORE', 'FIRST_PAYMENT_DATE', 'FIRST_TIME_HOMEBUYER', 'MATURITY_DATE', 'MSA', 'MI_PERCENT', 'NUMBER_OF_UNITS', \
              'OCCUPANCY_STATUS', 'ORIGINAL_CLTV', 'ORIGINAL_DTI', 'ORIGINAL_UPB', 'ORIGINAL_LTV', 'ORIGINAL_INTEREST_RATE', \
              'CHANNEL', 'PREPAYMENT_PENALTY_MORTGAGE', 'AMORTIZATION_TYPE', 'PROPERTY_STATE', 'PROPERTY_TYPE', 'POSTAL_CODE', \
              'LOAN_NUMBER', 'LOAN_PURPOSE', 'ORIGINAL_LOAN_TERM', 'NUMBER_OF_BORROWERS', 'SELLER_NAME', 'SERVICER_NAME', \
              'SUPER_CONFORMING', 'PREHARP_LOAN_NUMBER', 'PROGRAM_INDICATOR', 'HARP_INDICATOR', 'PROPERTY_VALUATION_METHOD', \
              'INTEREST_ONLY_INDICATOR']

orig_data_type = {'HARP_INDICATOR':str, 'POSTAL_CODE': str, 'PREHARP_LOAN_NUMBER':str, 'PROGRAM_INDICATOR':str, \
                 'FIRST_PAYMENT_DATE':'Int32', 'FIRST_TIME_HOMEBUYER':str, 'MATURITY_DATE':'Int32', 'MSA':str, 'MI_PERCENT':np.int32,\
                 'ORIGINAL_LOAN_TERM':'Int16', 'NUMBER_OF_BORROWERS':'Int16', 'SUPER_CONFORMING':str, \
                 'PROPERTY_VALUATION_METHOD':'Int16', 'NUMBER_OF_UNITS':str}

years = range(1999, 2005)
qtrs = range(1,5)

In [6]:
# iterate over the Origination Files
for file in originationFiles:
    for y, q in [(y,q) for y in years for q in qtrs]:
        fileName = f'{file}{y}Q{q}.txt'
        print('Processing ' + fileName)
        loadDataFile(fileName, 'loans', con, orig_columns, orig_data_type)

Processing historical_data_excl_1999Q1.txt
Processing historical_data_excl_1999Q2.txt
Processing historical_data_excl_1999Q3.txt
Processing historical_data_excl_1999Q4.txt
Processing historical_data_excl_2000Q1.txt
Processing historical_data_excl_2000Q2.txt
Processing historical_data_excl_2000Q3.txt
Processing historical_data_excl_2000Q4.txt
Processing historical_data_excl_2001Q1.txt
Processing historical_data_excl_2001Q2.txt
Processing historical_data_excl_2001Q3.txt
Processing historical_data_excl_2001Q4.txt
Processing historical_data_excl_2002Q1.txt
Processing historical_data_excl_2002Q2.txt
Processing historical_data_excl_2002Q3.txt
Processing historical_data_excl_2002Q4.txt
Processing historical_data_excl_2003Q1.txt
Processing historical_data_excl_2003Q2.txt
Processing historical_data_excl_2003Q3.txt
Processing historical_data_excl_2003Q4.txt
Processing historical_data_excl_2004Q1.txt
Processing historical_data_excl_2004Q2.txt
Processing historical_data_excl_2004Q3.txt
Processing 

In [7]:
# fileName='historical_data_1999Q1.txt'
# df = pd.read_csv('data/' + fileName, delimiter='|', header=None, names=orig_columns, dtype = orig_data_type)
# df.info()
# # 
# # historical_data_excl_1999Q1.txt

In [8]:
# df["NUMBER_OF_UNITS"].value_counts()

In [9]:
# df[df["NUMBER_OF_UNITS"]=='.']

In [10]:
# fileName='historical_data_1999Q1.txt'
# df = pd.read_csv('data/' + fileName, delimiter='|', header=None, names=orig_columns, dtype = orig_data_type)
# df['NUMBER_OF_UNITS'] = df['NUMBER_OF_UNITS'] 
# df['NUMBER_OF_UNITS'] = np.where(df['NUMBER_OF_UNITS'] == '.', 99, df['NUMBER_OF_UNITS'])

In [11]:
# df.to_sql(name='loans', con=con, schema='mortgage', if_exists='append', index=False)

In [12]:
# fileName='historical_data_1999Q1.txt'
# df = loadDataFile(fileName, 'loans', con, orig_columns, orig_data_type)

In [13]:
# remitFiles = ['historical_data_excl_time_','historical_data_time_'] # 1999Q1

# remit_columns=['LOAN_NUMBER', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB', 'DELINQUENCY_STATUS', 'LOAN_AGE', \
#               'REMAINING_MONTHS_TO_MATURITY', 'REPURCHASE', 'MODIFICATION', 'ZERO_BALANCE_CODE', 'ZERO_BALANCE_EFFECTIVE_DATE', \
#               'CURRENT_INTEREST_RATE', 'CURRENT_DEFERRED_UPB', 'DUE_DATE_OF_LAST_PAID_INSTALLMENT', \
#               'MI_RECOVERIES', 'NET_SALES_PROCEEDS', 'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAINTENANCE_PRESERVATION_COSTS', \
#               'TAXES_AND_INSURANCE', 'MISCELLANEOUS_EXPENSES', 'ACTUAL_LOSS_CALCULATION', 'MODIFICATION_COST', 'STEP_MODIFICATION',\
#               'DEFERRED_PAYMENT_PLAN', 'ESTIMATED_LOAN_TO_VALUE', 'ZERO_BALANCE_REMOVAL_UPB', 'DELINQUENT_ACCRUED_INTEREST', \
#               'DELINQUENCY_DUE_TO_DISASTER', 'BORROWER_ASSISTANCE_STATUS_CODE']

# remit_data_type = {'MODIFICATION':str, 'REPURCHASE': str, 'DELINQUENCY_STATUS':str, 'LOAN_NUMBER':str, \
#              'MONTHLY_REPORTING_PERIOD':'Int32', 'STEP_MODIFICATION':str, 'LOAN_AGE':'Int16', 'REMAINING_MONTHS_TO_MATURITY':str,\
#                    'ZERO_BALANCE_CODE':'Int16', 'ZERO_BALANCE_EFFECTIVE_DATE':'Int32', 'DUE_DATE_OF_LAST_PAID_INSTALLMENT':'Int32', \
#                   }

# # NET_SALES_PROCEEDS has value of U

In [14]:
# fileName = 'historical_data_excl_time_1999Q1.txt'
# df = loadDataFile(fileName, 'remittance', con, remit_columns, remit_data_type)

In [15]:
# df[(df['REMAINING_MONTHS_TO_MATURITY'] == '***')]
# df[(df['REMAINING_MONTHS_TO_MATURITY'] == '.')]

In [16]:
# df.count()

In [17]:
# df[df['CURRENT_ACTUAL_UPB']>=200000].count()
# df[(df['CURRENT_ACTUAL_UPB']>=170000) & (df['CURRENT_ACTUAL_UPB']<200000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=150000) & (df['CURRENT_ACTUAL_UPB']<170000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=130000) & (df['CURRENT_ACTUAL_UPB']<150000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=120000) & (df['CURRENT_ACTUAL_UPB']<130000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=110000) & (df['CURRENT_ACTUAL_UPB']<120000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=100000) & (df['CURRENT_ACTUAL_UPB']<110000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=90000) & (df['CURRENT_ACTUAL_UPB']<100000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=80000) & (df['CURRENT_ACTUAL_UPB']<90000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=70000) & (df['CURRENT_ACTUAL_UPB']<80000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=60000) & (df['CURRENT_ACTUAL_UPB']<70000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=50000) & (df['CURRENT_ACTUAL_UPB']<60000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=40000) & (df['CURRENT_ACTUAL_UPB']<50000)].count()
# df[(df['CURRENT_ACTUAL_UPB']>=30000) & (df['CURRENT_ACTUAL_UPB']<40000)].count()
# df[(df['CURRENT_ACTUAL_UPB']<30000)].count()

In [18]:
# # iterate over the Remittance Files
# for file in remitFiles:
#     for y, q in [(y,q) for y in years for q in qtrs]:
#         fileName = f'{file}{y}Q{q}.txt'
#         print('Processing ' + fileName)
#         loadDataFile(fileName, 'remittance', con, remit_columns, remit_data_type)

In [19]:
con.close()

In [20]:
# data = pd.read_sql(f"select * from loans", con)
# data.head()

In [21]:
# data = pd.read_sql(f"select * from remittance", con)
# data.head()

In [ ]:
def readRemit(fileName):
    remit_columns=['LOAN_NUMBER', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB', 'DELINQUENCY_STATUS', 'LOAN_AGE', \
              'REMAINING_MONTHS_TO_MATURITY', 'REPURCHASE', 'MODIFICATION', 'ZERO_BALANCE_CODE', 'ZERO_BALANCE_EFFECTIVE_DATE', \
              'CURRENT_INTEREST_RATE', 'CURRENT_DEFERRED_UPB', 'DUE_DATE_OF_LAST_PAID_INSTALLMENT', \
              'MI_RECOVERIES', 'NET_SALES_PROCEEDS', 'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAINTENANCE_PRESERVATION_COSTS', \
              'TAXES_AND_INSURANCE', 'MISCELLANEOUS_EXPENSES', 'ACTUAL_LOSS_CALCULATION', 'MODIFICATION_COST', 'STEP_MODIFICATION',\
              'DEFERRED_PAYMENT_PLAN', 'ESTIMATED_LOAN_TO_VALUE', 'ZERO_BALANCE_REMOVAL_UPB', 'DELINQUENT_ACCRUED_INTEREST', \
              'DELINQUENCY_DUE_TO_DISASTER', 'BORROWER_ASSISTANCE_STATUS_CODE']
    remit_data_type = {'MODIFICATION':str, 'REPURCHASE': str, 'DELINQUENCY_STATUS':str, 'LOAN_NUMBER':str, \
             'MONTHLY_REPORTING_PERIOD':'Int32', 'STEP_MODIFICATION':str, 'LOAN_AGE':'Int16', #'REMAINING_MONTHS_TO_MATURITY':'Int16',\
                   'ZERO_BALANCE_CODE':'Int16', 'ZERO_BALANCE_EFFECTIVE_DATE':'Int32', 'DUE_DATE_OF_LAST_PAID_INSTALLMENT':'Int32'
                  }
    df = pd.read_csv('data/' + fileName, delimiter='|', header=None, names=remit_columns, dtype = remit_data_type)
    drop_columns = ['CURRENT_ACTUAL_UPB', 'LOAN_AGE',\
              'REMAINING_MONTHS_TO_MATURITY', 'REPURCHASE', 'MODIFICATION', 'ZERO_BALANCE_EFFECTIVE_DATE', \
              'CURRENT_INTEREST_RATE', 'CURRENT_DEFERRED_UPB', 'DUE_DATE_OF_LAST_PAID_INSTALLMENT', \
              'MI_RECOVERIES', 'NET_SALES_PROCEEDS', 'NON_MI_RECOVERIES', 'EXPENSES', 'LEGAL_COSTS', 'MAINTENANCE_PRESERVATION_COSTS', \
              'TAXES_AND_INSURANCE', 'MISCELLANEOUS_EXPENSES', 'ACTUAL_LOSS_CALCULATION', 'MODIFICATION_COST', 'STEP_MODIFICATION',\
              'DEFERRED_PAYMENT_PLAN', 'ESTIMATED_LOAN_TO_VALUE', 'ZERO_BALANCE_REMOVAL_UPB', 'DELINQUENT_ACCRUED_INTEREST', \
              'DELINQUENCY_DUE_TO_DISASTER', 'BORROWER_ASSISTANCE_STATUS_CODE']
    df.drop(drop_columns, inplace=True, axis=1) # only keep a couple of columns to keep memory usage low
    # only keep the most recent monthly reporting period record to get the delinquency status to keep memory usage low
    df.sort_values(['LOAN_NUMBER','MONTHLY_REPORTING_PERIOD'], ascending=[True, False], inplace=True)
    df.drop_duplicates(['LOAN_NUMBER'], keep='first', inplace=True)
    return df

In [ ]:
remitFiles = ['historical_data_excl_time_','historical_data_time_'] # 1999Q1
years = range(1999, 2004) # 2005
qtrs = range(1,5)

firstOne = True
# iterate over the Remittance Files
for file in remitFiles:
    for y, q in [(y,q) for y in years for q in qtrs]:
        fileName = f'{file}{y}Q{q}.txt'
        print('Processing ' + fileName)
        df1 = readRemit(fileName)
        # Dataframe gets too big for memory to handle
        # write out to csv
        df1.to_csv('data/remit.csv', header=False, index=False, mode='a')
#         if firstOne:
#             dfRemit = df1
#             firstOne = False
#         else:
#             dfRemit = pd.concat([dfRemit, df1])

End up appending time series data into csv file.